# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [3]:
boston_features.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [5]:
X = boston_features[['ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']]
y = boston_features['CRIM']

## Train test split

Perform a train-test-split with a test set of 0.20.

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
from sklearn.metrics import mean_squared_error

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Fit the model and apply the model to the make test set predictions

In [10]:
linreg.fit(X_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [12]:
y_hat_train=linreg.predict(X_train)
y_hat_test= linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [16]:
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = nmean_squared_error(y_test, y_hat_test)

print(train_mse,test_mse)

47.701165149813086 11.760372666817075


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [24]:
def kfolds(data, k):
    data = pd.DataFrame(data)
    rem = len(data)%k
    ss,r_list = (len(data) - rem)//k, [1]*rem
    [r_list.append(i) for i in [0]*(k-rem)]
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers           
    return [data.iloc[(i)*ss:(i+1)*ss+r_list[i]].copy() for i in range(k)]

### Apply it to the Boston Housing Data

In [20]:
# Make sure to concatenate the data again
# boston_features.columns
b_y = 'CRIM'
b_x = ['ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']

In [38]:
folds = kfolds(boston_features,10)
test_err = []
train_err = []
for i in range(10):
    test = folds[i]
    train = pd.concat([folds[k] for k in range(10) if k != i])
    linreg.fit(train[b_x],train[b_y])
    y_hat_train=linreg.predict(train[b_x])
    y_hat_test= linreg.predict(test[b_x])
    train_err.append(mean_squared_error(train[b_y], y_hat_train))
    test_err.append(mean_squared_error(test[b_y], y_hat_test))

In [46]:
print(np.mean(test_err), np.mean(train_err))

53.39241468563796 39.83467518189006


### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [ ]:
# test_errs = []
# train_errs = []
# k=5

# for n in range(k):
#     # Split in train and test for the fold
#     train = None
#     test = None
#     # Fit a linear regression model
    
#     #Evaluate Train and Test Errors

# # print(train_errs)
# # print(test_errs)

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [53]:
from sklearn.model_selection import cross_val_score
X = boston_features[b_x]
y= boston_features[b_y]
cv_10_results = cross_val_score(linreg, X, y, cv=10, scoring="neg_mean_squared_error")
cv_20_results = cross_val_score(linreg, X, y, cv=10, scoring="neg_mean_squared_error")
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")
print(cv_5_results)

[  -4.17078428  -12.14164016   -5.13913859 -101.63014112 -125.74380865]


Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [54]:
np.mean(cv_5_results),np.mean(cv_10_results),np.mean(cv_20_results)


(-49.76510255992743, -52.973148116373025, -52.973148116373025)

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!